In [85]:
import praw
import pandas as pd
from datetime import datetime, timedelta
import os
import psaw as ps
from dotenv import load_dotenv
import time


In [86]:
# Lade die .env-Datei
dotenv_loaded = load_dotenv("zugang_reddit.env")  # Falls die Datei anders heißt, anpassen
# Prüfe, ob die Datei geladen wurde
print(f".env geladen? {dotenv_loaded}")


.env geladen? True


In [87]:
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent=os.getenv("USER_AGENT")
)

print("Reddit API erfolgreich verbunden!")


Reddit API erfolgreich verbunden!


In [88]:
try:
    subreddit = reddit.subreddit("CryptoCurrency")
    for post in subreddit.hot(limit=5):
        print(f"Title: {post.title}, Score: {post.score}")
except Exception as e:
    print(f"Fehler beim Abrufen der Subreddit-Daten: {e}")


Title: Kraken Pay AMA with Jared Blake, $5000 USDG* in prizes up for grabs!, Score: 18
Title: Daily Crypto Discussion - January 29, 2025 (GMT+0), Score: 31
Title: Crypto will change you, Score: 661
Title: They say humor can be a coping mechanism for trauma., Score: 99
Title: $6,270,000,000 Worth of Seized Bitcoin Could Be Sold By UK To Pay Off Debts: Accounting Giant, Score: 418


In [89]:
cryptos = [
    ("Cardano", "ADA"),
    ("Solana", "SOL"),
    ("Binance Coin", "BNB"),
    ("XRP", "XRP"),
    ("Polkadot", "DOT"),
    ("Dogecoin", "DOGE")
]


In [90]:
subreddits = [
    "CryptoCurrency",  # Allgemeine Diskussionen über Kryptowährungen
    "CryptoMarkets",   # Diskussionen über den Kryptomarkt und Preisbewegungen
    "CryptoTrading",   # Fokus auf Trading-Strategien und Analysen
    "Altcoin",         # Diskussionen über Altcoins (alle Kryptowährungen außer Bitcoin)
    "DeFi",            # Decentralized Finance (DeFi) und Projekte
    "BitcoinBeginners",# Für Anfänger in der Krypto-Welt
    "cryptotechnology", # Fokus auf die zugrunde liegende Blockchain-Technologie
    "cryptocurrencies", # Allgemeine Diskussionen über Kryptowährungen
    "Satoshistreetsbets", # Krypto-Wetten und Spekulationen
    "Binance"        # Diskussionen über die Binance-Plattform  
]

Scraping Funktionen

In [ ]:
# Funktion, um Posts und Kommentare gezielt zu Kryptowährungen zu scrapen
def scrape_reddit(start_date, end_date, mode="initial"):
    start_timestamp = int(start_date.timestamp())  # Umwandlung in Unix-Zeit
    end_timestamp = int(end_date.timestamp())  # Umwandlung in Unix-Zeit

    posts = []
    comments = []
    post_ids = set()  # Vermeidung doppelter Post-IDs

    for crypto_name, crypto_symbol in cryptos:
        for subreddit_name in subreddits:
            subreddit = reddit.subreddit(subreddit_name)
            print(f"Suche nach {crypto_name} ({crypto_symbol}) in r/{subreddit_name}...")

            # Suche gezielt nach dem Namen und Symbol der Kryptowährung
            for search_term in [crypto_name, crypto_symbol]:
                for post in subreddit.search(query=search_term, sort="new", limit=1000):
                    if start_timestamp <= post.created_utc <= end_timestamp and post.id not in post_ids:
                        post_ids.add(post.id)
                        posts.append({
                            'crypto': crypto_name,
                            'search_term': search_term,
                            'subreddit': subreddit_name,
                            'post_id': post.id,
                            'title': post.title,
                            'author': str(post.author),
                            'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                            'score': post.score,
                            'num_comments': post.num_comments,
                            'selftext': post.selftext
                        })
                        print(f"Post gefunden: {post.title} (Suchbegriff: {search_term})")

                        # Kommentare sammeln
                        post.comments.replace_more(limit=0)
                        for comment in post.comments.list():
                            comments.append({
                                'post_id': post.id,
                                'comment_id': comment.id,
                                'author': str(comment.author),
                                'created_utc': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                                'score': comment.score,
                                'body': comment.body
                            })

    # In DataFrames umwandeln
    df_posts = pd.DataFrame(posts)
    df_comments = pd.DataFrame(comments)

    # DataFrames zurückgeben
    print(f"Scrape abgeschlossen: {len(df_posts)} Posts und {len(df_comments)} Kommentare gefunden.")
    return df_posts, df_comments

# Einmaliger Scrape für die letzten 3 Monate
start_of_period = datetime(2024, 11, 1)  # 1. November 2024 als `datetime`
now = datetime.now()  # Aktuelle Zeit als `datetime`
print("Starte den einmaligen Scrape für die letzten 3 Monate...")
df_posts_initial, df_comments_initial = scrape_reddit(start_of_period, now, mode="initial")

# Beispiel: Lokale Weiterverarbeitung
print("Daten können jetzt in der Pipeline bereinigt werden...")

Starte den einmaligen Scrape für die letzten 3 Monate...
Suche nach Cardano (ADA) in r/CryptoCurrency...
Post gefunden: Cardano Set to Achieve Full Decentralized Governance with Wednesday’s Plomin Hard Fork (Suchbegriff: Cardano)
Post gefunden: Understanding the "inverse r/cryptocurrency" trading strategy. Is this sub consistently wrong? How it works, how to exploit it, and the underlying reason driving it: (Suchbegriff: Cardano)
Post gefunden: Tuttle Capital Seeks SEC Approval for First-Ever Chainlink, Cardano, and Polkadot ETFs (Suchbegriff: Cardano)
Post gefunden: Why are there no solid projects built on top layer 1s like sui and solana? (Suchbegriff: Cardano)
Post gefunden: Replacing ETC (Suchbegriff: Cardano)
Post gefunden: Trimming my portfolio - advice much appreciated! (Suchbegriff: Cardano)
Post gefunden: Crypto Trader Predicts Massive Rallies for Chainlink (LINK) and Cardano (ADA) – Here Are His Targets - The Daily Hodl (Suchbegriff: Cardano)
Post gefunden: Charles Hoskinson 

Die Scraping-Funktion ausfuehren

In [92]:
def clean_data(df_posts, df_comments):
    # 1. Duplikate entfernen
    df_posts = df_posts.drop_duplicates(subset=["post_id"])
    df_comments = df_comments.drop_duplicates(subset=["comment_id"])
    
    # 2. Fehlende Werte behandeln
    df_posts['selftext'] = df_posts['selftext'].fillna('')  # Fehlende Posttexte auffüllen
    df_comments['body'] = df_comments['body'].fillna('')  # Fehlende Kommentare auffüllen
    
    # 3. Zeitstempel konvertieren
    df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'])
    df_comments['created_utc'] = pd.to_datetime(df_comments['created_utc'])
    
    # 4. Filterung nach Qualität (z. B. Spam entfernen)
    df_posts = df_posts[df_posts['score'] > 0]  # Posts mit negativem Score entfernen
    df_comments = df_comments[df_comments['score'] > 0]  # Kommentare mit negativem Score entfernen
    
    return df_posts, df_comments


In [94]:
# Bereinigen der Daten
df_posts_clean, df_comments_clean = clean_data(df_posts_initial, df_comments_initial)

# Überprüfen, wie viele Einträge übrig sind
print(f"Bereinigte Posts: {len(df_posts_clean)}")
print(f"Bereinigte Kommentare: {len(df_comments_clean)}")

Bereinigte Posts: 988
Bereinigte Kommentare: 60423


In [100]:
df_posts_clean

,crypto,search_term,subreddit,post_id,title,author,created_utc,score,num_comments,selftext
0,Cardano,Cardano,CryptoCurrency,1iczhdd,Cardano Set to Achieve Full Decentralized Gove...,Amazonreviewscool67,2025-01-29 17:36:48,13,3,
1,Cardano,Cardano,CryptoCurrency,1ic4dus,"Understanding the ""inverse r/cryptocurrency"" t...",fan_of_hakiksexydays,2025-01-28 15:42:12,1,26,"We've all heard the jokes, similar to how inve..."
2,Cardano,Cardano,CryptoCurrency,1iby3mf,Tuttle Capital Seeks SEC Approval for First-Ev...,kirtash93,2025-01-28 09:49:43,5,16,
5,Cardano,Cardano,CryptoCurrency,1i7i9zk,Trimming my portfolio - advice much appreciated!,TomekGregory,2025-01-22 18:37:20,2,19,"Hi, \nI currently have exposure to the coins ..."
6,Cardano,Cardano,CryptoCurrency,1i5n661,Crypto Trader Predicts Massive Rallies for Cha...,PeterParkerUber,2025-01-20 10:20:25,1,13,
...,...,...,...,...,...,...,...,...,...,...
1541,Dogecoin,DOGE,DeFi,1gpwkkv,Pepe coin price prediction of 2025,fegadi,2024-11-12 21:51:13,20,59,Yo fellas what do you think Pepe coin could re...
1542,Dogecoin,Dogecoin,cryptotechnology,1hl8032,Can a scammer set up something to immediately ...,Chakasicle,2024-12-24 06:54:54,15,28,Im new to crypto and trying to figure out if I...
1543,Dogecoin,Dogecoin,cryptocurrencies,1ichk6q,"New, looking for a solid ASIC Machine around 5...",OldOneEye_Tien,2025-01-29 00:54:24,1,1,"Title says most of it, I have looked all over ..."
1544,Dogecoin,Dogecoin,cryptocurrencies,1i7ud2p,Bitwise Files For World’s First Dogecoin ETF i...,Traditional_Motor_51,2025-01-23 03:33:03,3,0,


In [101]:
# Definiere den Speicherpfad zu deinem lokalen Google Drive
DRIVE_PATH = "G:/Meine Ablage/"


In [105]:
def append_to_csv(df_new, filename):
    file_path = os.path.join(DRIVE_PATH, filename)

    try:
        # Falls die Datei bereits existiert, lese sie ein und hänge die neuen Daten an
        if os.path.exists(file_path):
            df_existing = pd.read_csv(file_path, sep=";", encoding="utf-8-sig", on_bad_lines="skip")
            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        else:
            df_combined = df_new  # Falls die Datei nicht existiert, erstelle sie neu

        # CSV speichern mit besserer Formatierung für Excel
        df_combined.to_csv(
            file_path,
            index=False,
            sep=";",  # Semikolon als Trennzeichen für bessere Excel-Kompatibilität
            encoding="utf-8-sig",  # Richtige Zeichenkodierung für Excel
            lineterminator="\n"  # Zeilenumbrüche korrekt speichern
        )
        print(f"Datei erfolgreich aktualisiert: {file_path}")

    except Exception as e:
        print(f"Fehler beim Speichern der Datei {filename}: {e}")

def export_to_drive(df_posts, df_comments):
    try:
        # Posts an bestehende Datei anhängen
        append_to_csv(df_posts, "reddit_posts.csv")
        # Kommentare an bestehende Datei anhängen
        append_to_csv(df_comments, "reddit_comments.csv")

    except Exception as e:
        print(f"Fehler beim Export: {e}")


In [106]:
# Export der bereinigten Daten
export_to_drive(df_posts_clean, df_comments_clean)

Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit_posts.csv
Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit_comments.csv
